In [1]:
!pip install pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark
from pyspark.sql import functions as F

In [2]:
# Выгрузка CSV с автоподбором типа
def load_csv(spark, file_path):
  return spark.read.option("header", "true").option("inferSchema", "true").csv(file_path)

# Инициализация сессии
def init(session_name):
    return SparkSession.builder \
        .appName(session_name) \
        .config("spark.sql.legacy.timeParserPolicy", "LEGACY") \
        .getOrCreate()


spark, trips_df, stations_df = init("lr2"), load_csv(spark, "trip.csv"), load_csv(spark, "station.csv")

print(stations_df.dtypes, '\n', trips_df.dtypes, sep='\n')

[('id', 'int'), ('name', 'string'), ('lat', 'double'), ('long', 'double'), ('dock_count', 'int'), ('city', 'string'), ('installation_date', 'string')]


[('id', 'int'), ('duration', 'int'), ('start_date', 'string'), ('start_station_name', 'string'), ('start_station_id', 'int'), ('end_date', 'string'), ('end_station_name', 'string'), ('end_station_id', 'int'), ('bike_id', 'int'), ('subscription_type', 'string'), ('zip_code', 'string')]


In [3]:
# Задание 1. Найти велосипед с максимальным временем пробега

result = (
    trips_df.withColumn("duration_minutes",
                        (F.unix_timestamp("end_date", "M/d/yyyy H:mm") -
                         F.unix_timestamp("start_date", "M/d/yyyy H:mm")) / 3600)
    .groupBy("bike_id")
    .agg(F.sum("duration_minutes").alias("total_duration_minutes"))
    .orderBy(F.desc("total_duration_minutes"))
    .limit(1)
).collect()[0]

print(f"Ответ: Велосипед с ID {result.bike_id} имеет максимальное время пробега: {int(result.total_duration_minutes)} часов")


Ответ: Велосипед с ID 535 имеет максимальное время пробега: 5171 часов


In [4]:
# Задание 2. Найти наибольшее геодезическое расстояние между станциями.

from pyspark.sql.functions import max, radians, sin, cos, acos, col

# Функция для вычисления геодезического расстояния между двумя точками
def dist_calc(lat1, lon1, lat2, lon2):

    lat1, lat2, lon1, lon2 = radians(lat1), radians(lat2), radians(lon1), radians(lon2)
    return 6371 * acos(sin(lat1) * sin(lat2) + cos(lat1) * cos(lat2) * cos(lon2 - lon1))

max_distance = (stations_df.alias("station1")
                .crossJoin(stations_df.alias("station2"))
                .filter(col("station1.id") < col("station2.id"))
                .withColumn("distance", dist_calc(col("station1.lat"), col("station1.long"), col("station2.lat"), col("station2.long")))
                .agg(max("distance").alias("max_distance"))
                .collect()[0]["max_distance"])

print(f"Ответ: Максимальное геодезическое расстояние между станциями: {max_distance:.6f} км")

Ответ: Максимальное геодезическое расстояние между станциями: 69.920876 км


In [5]:
# Задание 3. Найти путь велосипеда с максимальным временем пробега через станции.
result = (
    trips_df.withColumn(
        "duration_minutes",
        (F.unix_timestamp("end_date", "M/d/yyyy H:mm") -
         F.unix_timestamp("start_date", "M/d/yyyy H:mm")) / 60
        )
    .groupBy("bike_id")
    .agg(F.sum("duration_minutes").alias("total_duration_minutes"))
    .orderBy(F.desc("total_duration_minutes"))
    .limit(1)
)


max_bike_duration = result.collect()

if max_bike_duration:
    bike_id_max = max_bike_duration[0]["bike_id"]
    total_duration = max_bike_duration[0]["total_duration_minutes"]

    # Отбираем поездки для этого велосипеда и сортируем по времени начала
    bike_trips = trips_df.filter(F.col("bike_id") == bike_id_max) .select("start_date", "start_station_name", "end_date", "end_station_name").orderBy("start_date")
    bike_trips.show(truncate=False)


+---------------+---------------------------------------------+---------------+---------------------------------------------+
|start_date     |start_station_name                           |end_date       |end_station_name                             |
+---------------+---------------------------------------------+---------------+---------------------------------------------+
|1/1/2014 13:42 |Mechanics Plaza (Market at Battery)          |1/1/2014 14:36 |Embarcadero at Sansome                       |
|1/1/2014 18:51 |Embarcadero at Sansome                       |1/1/2014 19:13 |Market at 4th                                |
|1/1/2014 19:48 |Market at 4th                                |1/1/2014 20:01 |South Van Ness at Market                     |
|1/10/2014 20:13|Market at 10th                               |1/10/2014 20:17|Powell Street BART                           |
|1/10/2014 8:09 |Embarcadero at Folsom                        |1/10/2014 8:19 |San Francisco Caltrain (Townsend at 4th

In [6]:
# Задание 4. Найти количество велосипедов в системе.

total_bikes = trips_df.select("bike_id").distinct().count()
print(f"Количество уникальных велосипедов в системе: {total_bikes}")


Количество уникальных велосипедов в системе: 700


In [9]:
# Задание 5. Найти пользователей потративших на поездки более 3 часов.

trips_with_duration = trips_df.withColumn(
    "trip_hours",
    (F.unix_timestamp("end_date", "M/d/yyyy H:mm") -
     F.unix_timestamp("start_date", "M/d/yyyy H:mm")) / 3600
)

user_trip_summary = trips_with_duration.groupBy("zip_code").agg(
    F.sum("trip_hours").alias("hours")
)
active_users = user_trip_summary.filter(F.col("hours") > 3)

active_users.select("zip_code", "hours").show()


+--------+------------------+
|zip_code|             hours|
+--------+------------------+
|   94102| 5312.433333333343|
|   95134|201.89999999999984|
|   84606|26.383333333333333|
|   80305| 50.16666666666665|
|   60070|              8.05|
|   95519| 8.416666666666666|
|   43085|3.2333333333333334|
|   91910|13.999999999999998|
|   77339|3.8333333333333335|
|   48063|               3.8|
|   85022|3.4833333333333334|
|    1090| 5.666666666666666|
|    2136| 4.433333333333334|
|   11722|              6.75|
|   95138|             43.05|
|   94610|1008.1666666666683|
|   94404| 996.8833333333289|
|   80301| 42.28333333333333|
|   91326|18.266666666666666|
|   90742|3.0500000000000003|
+--------+------------------+
only showing top 20 rows

